# Test your numpy knowledge

The following programming problems are based on some of the things you learn in the respective sections of this module. If you struggle solving any problem, you might want to check out the section.

In [ ]:
import numpy as np

## Section 1: Array Creation, Indexing, and Slicing
Create a `ndarray` that corresponds to the following list-of-list representation. Do so by only using functions available in plain numpy and without using `np.array(...)` or `np.load(...)` to load the image into numpy.

In [ ]:
[[1., 1., 1., 1., 1., 6., 1., 1., 1., 1., 1.],
 [1., 2., 2., 2., 2., 6., 2., 2., 2., 2., 1.],
 [1., 2., 3., 3., 3., 6., 3., 3., 3., 2., 1.],
 [1., 2., 3., 4., 4., 6., 4., 4., 3., 2., 1.],
 [1., 2., 3., 4., 5., 6., 5., 4., 3., 2., 1.],
 [6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
 [1., 2., 3., 4., 5., 6., 5., 4., 3., 2., 1.],
 [1., 2., 3., 4., 4., 6., 4., 4., 3., 2., 1.],
 [1., 2., 3., 3., 3., 6., 3., 3., 3., 2., 1.],
 [1., 2., 2., 2., 2., 6., 2., 2., 2., 2., 1.],
 [1., 1., 1., 1., 1., 6., 1., 1., 1., 1., 1.]]

<details>
<summary>Answers (click me to reveal)</summary>

```
foo = np.ones((11,11))
for idx in range(6):
    foo[idx:-idx, idx:-idx] = idx + 1
foo[5] = 6
foo[:, 5] = 6
```

</details>

## Section 2: Vectorization and the Functional API

The array `weather_data` contians daily Swedish weather data from Jan-2020 until approximately the end of Apr-2020. Each row in the data matrix corresponds to a day, and each column in the data matrix contains a different feature of the weather. In more detail, the four columns contain `[temperatureLow, temperatureHigh, humidity, windSpeed]` in that order. Unfortunately, the temperature is measured in °F (Farenheit) instead of °C (Celsius).

Compute the minimum and maximum temperature in Celsius (°C) for each day. Then compute the overall average temperature as well as the temperature's standard deviation.

In [ ]:
weather_data = np.load("weather_data.npy")

<details>
<summary>Answer (click me to reveal)</summary>

Exercise 3
```
weather_converted = weather_data.copy()
temp_in_C = (weather_data[:, :2] - 32 ) * 5/9
weather_converted[:, :2] = temp_in_C

mean_temperature = np.mean(temp_in_C)
standard_deviation = np.std(temp_in_C)
```
</details>

## Section 3: Broadcasting and Fancy Indexing

`vectorA` contains position measurements of a person walking down a (1D) corridor. Samples are taking at a rate of 100Hz and the vector stores 10 seconds worth data. Unfortunatley, the data is quite noisy (accuracy ~ 10 cm). To improve accuracy you would like to use your a priori knowledge that people tend to not teleport and apply a moving average filter. You decide that 0.1s is a reasonable window size for this scenario. 

In order to easily apply this filter in a vectorized fashion, you need to reorganize the data stored in `vectorA`. Create a batch of vectors (`vector_batch`, a matrix) that, in each row, stores 0.1 seconds worth of position data, and where the column corresponds to progressing time. That is, the first row of the matrix contains the measurements from 0s to 0.1s, the second row contains the measurements from 0.01s to 0.11s, the third row contains the measurements from 0.02s to 0.62s, and so on until the last row, which contains measurements from 9.9s to 10s. (Hint: Use broadcasting to create a matrix containing the required indices and then use fancy indexing to retrieve the actual measurements.)

After you have created `vector_batch`, apply the rolling average filter by computing the average value of each row.

In [ ]:
vectorA = np.load("position_data.npy")

<details>
<summary>Answer (click me to reveal)</summary>

```
# at 100Hz 50 samples make up 0.5s
window = np.arange(10)

# a new window starts at every sample that has at least len(window) samples following it
row_idx = np.arange(len(vectorA) - len(window))  

# matrix of sample indices via broadcasting
batch_idxs = row_idx[:, None] + window[None, :]

# fancy-indexing to retrieve the values from the indices
vector_batch = vectorA[batch_idxs]

# filtering
rolling_average = np.mean(vector_batch, axis=-1)

# Bonus: new accuracy for this scenario
vectorA_true = np.linspace(0, 5, 5*100)
absolute_error = np.abs(vectorA_true[len(window):] - rolling_average)
print(f"The accuracy of the new sequence is ~ {np.mean(absolute_error)*100:.2f} cm")
```

</details>

## Section 4: Memory Layout and Views

What is the value of `result` in the following program?

In [ ]:
foo = np.ones(9)
bar = foo.reshape(3,3)
bar += np.array([1, 2, 3])[:, None]
baz = bar.ravel()
baz[7] = 2

result = np.sum(foo[-3:])

<details>
<summary>Answer (click me to reveal)</summary>

The answer is 10. `foo.reshape` produces a view into `foo`, and all modifications to bar will also affect foo. Similarly, baz is a view into bar, which again shares data with foo. After the dust has settled from the modifications, foo looks like this `[2., 2., 2., 3., 3., 3., 4., 2., 4.]` and summing the last three elements is 4 + 2 + 4 = 10.

</details>